# Didital Garage and Naviplus 主催

# 第5回「Python で体験する機械学習」勉強会

<center>
<span style="color:Forestgreen">浅川伸一</span> 
<span style="color:SpringGreen">asakawa@ieee.org</span>
</center>

> 過去が現在に影響を与えるように、未来も現在に影響を与える。
> ニーチェ

In [1]:
#!/bin/env/python
# -*- encoding: utf-8 -*-
from __future__ import print_function
import numpy as np
from six.moves import xrange

In [40]:
# 入力データの定義
X = np.array([[0,0,1],[0,1,1],[1,0,1],[1,1,1]])

# 教師データの定義
y = np.array([[0,1,1,0]]).T

# 乱数系列の初期化
np.random.seed(seed=1)

# 結合係数行列の初期化
Win  = 2 * np.random.random((3,4)) - 1
Wout = 2 * np.random.random((4,1)) - 1

# 学習ループ
for i in  range(1000):
    hidden = 1/(1+np.exp(-(np.dot(X,Win))))
    out    = 1/(1+np.exp(-(np.dot(hidden,Wout))))
    delta_out = ( y -  out ) * ( out * ( 1 - out ))
    delta_hid = delta_out.dot(Wout.T) * ( hidden * (1 - hidden))
    Wout += hidden.T.dot(delta_out)
    Win  += X.T.dot(delta_hid)

# 結果の表示
print(out)

[[ 0.03508131]
 [ 0.95463069]
 [ 0.96195439]
 [ 0.05360758]]



![image](./assets/xor-graph.svg){!#fig:xor-graph style="width:24%"}
![image](./assets/xor.svg){!#fig:xor style="width:9%"}

---
# binary addition

In [11]:
#!/usr/bin/env python
# -*- encoding: utf-8 -*-
from __future__ import print_function
""" https://www.youtube.com/watch?v=cdLUzrjnlr4"""
"""i_rnn.py: test rnn code"""
from six.moves import xrange
import numpy as np
import sys
import copy    # for deepcopy

In [12]:
# シグモイド関数の定義
def sigmoid(x):
    return 1./(1.+np.exp(-x))

# シグモイド関数の微分
def d_sigmoid(x):
    return x * (1.-x)

In [13]:
# 2 進数のデータを作成
int2binary = {}
binary_dim = 7

largest_number = pow(2,binary_dim)
binary = np.unpackbits(
    np.array([range(largest_number)], dtype=np.uint8).T, axis=1)

for i in range(largest_number):
    int2binary[i] = binary[i]

In [14]:
# ハイパーパラメータの設定
np.random.seed(seed=1)
lr = 0.1             # 学習率 learning ratio
MaxIter = 10000      # 繰り返しの最大数
N_i = 2              # 入力層のニューロン数 (入力次元数)
N_h = 8              # 中間層のニューロン数 (可変)
N_o = 1              # 出力層のニューロン数     

# 結合係数行列の初期化
W_ih = 2 * np.random.random((N_i, N_h)) - 1.
W_ho = 2 * np.random.random((N_h, N_o)) - 1.
W_hh = 2 * np.random.random((N_h, N_h)) - 1.

# 学習に用いる差分行列の定義
dW_ih = np.zeros_like(W_ih)
dW_ho = np.zeros_like(W_ho)
dW_hh = np.zeros_like(W_hh)

### 以下のセルは実行と関係なし

In [17]:
### 以下のループの意味
a = int2binary[3]
b = int2binary[4]
X = np.array([[a, b]])
print('X=',X)
c = int2binary[7]
# print(c)
Y = np.array([c]).T
print('Y=', Y)
print('for loop counter:', binary_dim)

X= [[[0 0 0 0 0 0 1 1]
  [0 0 0 0 0 1 0 0]]]
Y= [[0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]]
for loop counter: 7


## 以下のセルは実行と関係なし

## 以下のセルは実行と関係なし

In [19]:
for position in range(binary_dim):
    print(binary_dim - position - 1, end=' ')
print()    

6 5 4 3 2 1 0 


In [18]:
# 具体例
X = np.array([[a[3], b[3]]])
X

array([[0, 0]], dtype=uint8)

In [16]:
for iter in range(MaxIter):
    # 任意の数を生成して a に代入
    a_int = np.random.randint(largest_number / 2)
    a = int2binary[a_int]

    # 任意の数を生成して b に代入
    b_int = np.random.randint(largest_number / 2)
    b = int2binary[b_int]

    # a + b の答えを c に代入
    c_int = a_int + b_int
    c = int2binary[c_int]

    d = np.zeros_like(c)

    total_err = 0
    Output_deltas = list()
    Hidden_values = list()
    Hidden_values.append(np.zeros(N_h))

    # forward prop
    for position in range(binary_dim):
        # a, b の上位桁から取り出して X に代入
        X = np.array([[a[binary_dim - position - 1], b[binary_dim - position - 1]]])
        # y に c の上位桁から代入
        y = np.array([[c[binary_dim - position - 1]]]).T

        # 入力行列 X に W_ih をかけて，Hidden_values かける W_hh をたして
        Hidden = sigmoid(np.dot(X, W_ih) + np.dot(Hidden_values[-1],W_hh))
        Output = sigmoid(np.dot(Hidden, W_ho))

        Delta = y - Output
        Output_deltas.append((Delta) * d_sigmoid(Output))
        total_err += np.abs(Delta[0])

        ### prediction
        d[binary_dim - position - 1] = np.round(Output[0][0])
        Hidden_values.append(copy.deepcopy(Hidden))
        #print('position=%d' % (position), end=' ')
        #print(Hidden_values)

    future_Hidden_delta = np.zeros(N_h)

    # back prop
    for position in range(binary_dim):
        X = np.array([[a[position], b[position]]])
        Hidden       = Hidden_values[-position - 1]
        prev_Hidden  = Hidden_values[-position - 2]

        Output_delta = Output_deltas[-position - 1]
        Hidden_delta = (future_Hidden_delta.dot(W_hh.T) + Output_delta.dot(W_ho.T)) * d_sigmoid(Hidden)

        dW_ho += np.atleast_2d(     Hidden).T.dot(Output_delta)
        dW_hh += np.atleast_2d(prev_Hidden).T.dot(Hidden_delta)
        dW_ih += X.T.dot(Hidden_delta)
        future_Hiddne_delta = Hidden_delta

    W_ho += lr * dW_ho
    W_hh += lr * dW_hh
    W_ih += lr * dW_ih

    dW_ho *= 0
    dW_hh *= 0
    dW_ih *= 0

    if iter % 1000 == 0:
        print("iteration=",iter)
        print("Error: %f" % (total_err))
        print("Pred: %s" % (str(d)))
        print("True: %s" % (str(c)))
        out = 0
        for index, x in enumerate(reversed(d)):
            out += x * pow(2,index)
        print(str(a_int) + " + " + str(b_int) + " = " + str(out))


iteration= 0
Error: 3.733012
Pred: [1 1 1 1 1 1 1 0]
True: [0 0 1 0 0 0 1 1]
33 + 2 = 254
iteration= 1000
Error: 3.150879
Pred: [0 0 0 0 0 0 0 0]
True: [0 1 0 1 0 0 1 1]
51 + 32 = 0
iteration= 2000
Error: 3.021493
Pred: [0 1 1 1 1 1 1 0]
True: [0 1 0 1 1 1 1 1]
47 + 48 = 126
iteration= 3000
Error: 3.043830
Pred: [0 0 1 1 1 0 1 0]
True: [0 1 0 1 1 0 1 1]
61 + 30 = 58
iteration= 4000
Error: 3.055076
Pred: [0 1 1 1 1 0 0 0]
True: [0 1 0 0 0 0 0 1]
12 + 53 = 120
iteration= 5000
Error: 2.757266
Pred: [0 1 0 1 1 0 1 0]
True: [0 1 0 1 0 0 1 1]
46 + 37 = 90
iteration= 6000
Error: 2.688775
Pred: [0 1 1 0 0 1 0 0]
True: [0 1 0 0 1 1 0 0]
28 + 48 = 100
iteration= 7000
Error: 2.060450
Pred: [0 1 0 0 1 0 1 0]
True: [0 1 0 0 1 0 1 0]
34 + 40 = 74
iteration= 8000
Error: 3.562014
Pred: [0 0 1 0 1 1 1 0]
True: [0 1 0 0 0 0 0 0]
23 + 41 = 46
iteration= 9000
Error: 1.881506
Pred: [0 0 1 1 1 0 1 0]
True: [0 0 1 1 1 0 1 0]
20 + 38 = 58


---

# sequential mnist

'''
A Reccurent Neural Network (LSTM) implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/)
Long Short Term Memory paper: http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

In [7]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/", one_hot=True)

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz


In [8]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [9]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Permuting batch_size and n_steps
    x = tf.transpose(x, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.split(x, n_steps, 0)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [10]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Iter 1280, Minibatch Loss= 1.812274, Training Accuracy= 0.35156
Iter 2560, Minibatch Loss= 1.503676, Training Accuracy= 0.56250
Iter 3840, Minibatch Loss= 1.173299, Training Accuracy= 0.58594
Iter 5120, Minibatch Loss= 0.976452, Training Accuracy= 0.64844
Iter 6400, Minibatch Loss= 0.719587, Training Accuracy= 0.76562
Iter 7680, Minibatch Loss= 1.163240, Training Accuracy= 0.59375
Iter 8960, Minibatch Loss= 0.764033, Training Accuracy= 0.75000
Iter 10240, Minibatch Loss= 0.645423, Training Accuracy= 0.79688
Iter 11520, Minibatch Loss= 0.369868, Training Accuracy= 0.94531
Iter 12800, Minibatch Loss= 0.572701, Training Accuracy= 0.79688
Iter 14080, Minibatch Loss= 0.547925, Training Accuracy= 0.83594
Iter 15360, Minibatch Loss= 0.387878, Training Accuracy= 0.89062
Iter 16640, Minibatch Loss= 0.358034, Training Accuracy= 0.93750
Iter 17920, Minibatch Loss= 0.328301, Training Accuracy= 0.90625
Iter 19200, Minibatch Loss= 0.340854, Training Accuracy= 0.86719
Iter 20480, Minibatch Loss= 0.17